<a href="https://colab.research.google.com/github/1ArjunPandey/RAG_Application/blob/master/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd

csv_file = '/content/query_answer_dataset.csv'
dataset = pd.read_csv(csv_file)

In [ ]:
dataset

,Query,Answer
0,What are the general principles of procurement...,The general principles include adherence to hi...
1,What does Rule 21 of GFR 2017 emphasize?,Rule 21 of GFR 2017 emphasizes that every offi...
2,How should an officer exercise vigilance in re...,An officer should exercise the same vigilance ...
3,What principle applies to the scale of expendi...,The expenditure should not be prima facie more...
4,Can an authority sanction expenditure that ben...,No authority should exercise its powers of san...
5,Under what conditions can expenditure be incur...,Expenditure for a particular person or section...
6,What is the primary responsibility of authorit...,Their primary responsibility is to bring effic...
7,What are the key factors to consider when spec...,"Key factors include clearly defining quality, ..."
8,How should offers be invited in the procuremen...,"Offers should be invited following a fair, tra..."
9,What is the purpose of competitive bidding in ...,Competitive bidding is adopted to ensure fair ...


# dATA

In [ ]:
import pandas as pd


# Define the alpaca_prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

# Load your dataset from the CSV file
csv_file = '/content/query_answer_dataset.csv'
dataset = pd.read_csv(csv_file)

# Extract the 'Query' and 'Answer' columns
instructions = dataset["Query"].tolist()
outputs = dataset["Answer"].tolist()


EOS_TOKEN = tokenizer.eos_token

# Formatting function to create the "text" column
def formatting_prompts_func(instructions, outputs):
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Create the formatted text and add EOS_TOKEN
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return texts

# Generate the formatted text column
dataset['text'] = formatting_prompts_func(instructions, outputs)

In [ ]:
dataset

,Query,Answer,text
0,What are the general principles of procurement...,The general principles include adherence to hi...,"Below is an instruction that describes a task,..."
1,What does Rule 21 of GFR 2017 emphasize?,Rule 21 of GFR 2017 emphasizes that every offi...,"Below is an instruction that describes a task,..."
2,How should an officer exercise vigilance in re...,An officer should exercise the same vigilance ...,"Below is an instruction that describes a task,..."
3,What principle applies to the scale of expendi...,The expenditure should not be prima facie more...,"Below is an instruction that describes a task,..."
4,Can an authority sanction expenditure that ben...,No authority should exercise its powers of san...,"Below is an instruction that describes a task,..."
5,Under what conditions can expenditure be incur...,Expenditure for a particular person or section...,"Below is an instruction that describes a task,..."
6,What is the primary responsibility of authorit...,Their primary responsibility is to bring effic...,"Below is an instruction that describes a task,..."
7,What are the key factors to consider when spec...,"Key factors include clearly defining quality, ...","Below is an instruction that describes a task,..."
8,How should offers be invited in the procuremen...,"Offers should be invited following a fair, tra...","Below is an instruction that describes a task,..."
9,What is the purpose of competitive bidding in ...,Competitive bidding is adopted to ensure fair ...,"Below is an instruction that describes a task,..."


In [ ]:
for i in range(3):
  print(i ,"th data")
  print(dataset['text'][i])
  print("\n")

0 th data
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What are the general principles of procurement according to the manual?

### Response:
The general principles include adherence to high standards of financial propriety, making procurements only in cases of proven necessity, and ensuring efficiency and economy in the procurement process.<|end_of_text|>


1 th data
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What does Rule 21 of GFR 2017 emphasize?

### Response:
Rule 21 of GFR 2017 emphasizes that every officer incurring or authorizing expenditure from public money should be guided by high standards of financial propriety and enforce financial order and strict economy.<|end_of_text|>


2 th data
Below is an instruction that

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)

In [ ]:
dataset

Dataset({
    features: ['Query', 'Answer', 'text'],
    num_rows: 50
})

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.215500
2,3.462900
3,3.424800
4,2.998900
5,2.935500
6,2.568000
7,2.148500
8,1.910300
9,1.551700
10,1.637800


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What does Rule 21 of GFR 2017 emphasize? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Rule 21 of GFR 2017 emphasizes that every officer incurring or authorizing expenditure from
#  public money should be guided by high standards of financial propriety and enforce financial order and strict economy.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What does Rule 21 of GFR 2017 emphasize? 

### Response:
 Rule 21 of GFR 2017 emphasizes that no expenditure shall be incurred on an item of supply or service, which is not authorized or is beyond the limit prescribed for the purpose.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " How is Revenue Procurement defined in the manual?", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Revenue Procurement implies the procurement of items and equipment to maintain
# and operate already sanctioned assets, including replacement equipment and components.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 How is Revenue Procurement defined in the manual?

### Response:
 Revenue Procurement is defined as any procurement of goods or services that involves the expenditure of public money from the Revenue account.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What is the retention period for procurement records? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Records should be retained for five years beyond completion of contractual obligations or project closure, whichever is applicable and later.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What is the retention period for procurement records? 

### Response:
 Records should be retained for at least five years beyond completion or closure of the procurement process.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What is the purpose of competitive bidding in procurement?", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


# Competitive bidding is adopted to ensure fair competition and to realize the best value for money from the best acceptable bidder.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What is the purpose of competitive bidding in procurement?

### Response:
 Competitive bidding ensures that the best price is obtained for the goods or services required.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What penalties can be imposed for violating the Code of Integrity? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Penalties can include exclusion from the procurement process, forfeiture of securities, cancellation of contracts, and debarment from future procurements.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What penalties can be imposed for violating the Code of Integrity? 

### Response:
 Violation of the Code can result in exclusion from participation in future procurements, cancellation of the present contract, or debarment from further business with the Government.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " Can an authority sanction expenditure that benefits itself?", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

# No authority should exercise its powers of sanctioning expenditure to
# pass an order that will be directly or indirectly to its own advantage.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 Can an authority sanction expenditure that benefits itself?

### Response:
 An authority may not sanction expenditure that benefits itself or any of its officers or servants or any person related to any of them.<|end_of_text|>
